In [4]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
print(tf.__version__)
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.model_selection import train_test_split
seed = 7
np.random.seed(seed)


2.16.1


## Types of Model APIs in Keras 
There are three main ways of creating models in Keras.
* **The Sequential Model**: This is a straightforward way of stacking layers but is limited to having a single-input and single-output stack of layers. 
* **The Functional API**: This can support arbitrary model architectures, and is more flexible in enabling complex implementations.

* **Model subclassing**: This is a way of implementing models from scratch. This is mainly used for out-of-the-box research use cases.


In [5]:
dataframe = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/data/sonar.csv", header=None)

In [7]:
dataset = dataframe.values

X = dataset[:, 0:60].astype(float)
y = dataset[:, 60]

# encode labels
le = LabelEncoder()
encoded_y = le.fit(y).transform(y)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, encoded_y, test_size=0.20, random_state=42)

### **The Sequential Model API** 
The Sequential model API groups a linear stack of layers into a `tf.keras.Model`. It doesn't allow you to create models that share layers, have branches, multiple inputs, or multiple outputs. It does provide training and inference features on this model.  

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/sequential_model.png" alt="The Sequential model API" width="50%">




As you'll see, using the Sequential API is simple and straightforward, but is only appropriate for simpler, more straightforward tasks. Later in this notebook you'll spend some time building with a more flexible, powerful alternative: the Functional API.


### Create the Sequential ModelWe start off by initializing the model as an instance of the Sequential class, like so: `model = Sequential()`. We follow this by adding layers, normalization layers, methods, softmax classifiers, and so on to the class, one at a time, using `model.add()`. This can be used to add different types of layer instances on top of the layer stack. Similarly, we can use `model.pop()` to remove the last layer of the model. 

Typically, the first layer receives an `input_shape` argument. This is because, in Keras, the shape of the weights is based on the shape of the inputs. The weights are only created when the model first sees some input data. After that, we can omit this argument and automate the inference of the input shape.

Note that if your model is non-linear or contains layers with multiple inputs or outputs, a Sequential model wouldn't be the right choice. 



In [13]:
def baseline_model():
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape = (60, )))
    model.add(Dense(60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [14]:
estimator = baseline_model()

In [15]:
estimator.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                      │ (None, 60)                  │           3,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 60)                  │           3,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              61 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,381 (28.83 KB)

 Trainable params: 7,381 (28.83 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
estimator.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8052 - loss: 0.4949 
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8376 - loss: 0.4503 
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8261 - loss: 0.4431 
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8220 - loss: 0.4301 
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8550 - loss: 0.4192 
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8738 - loss: 0.3895 
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8303 - loss: 0.4000 
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8608 - loss: 0.3852 
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8577 - loss: 0.3828 
Epoch 10/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9084 - loss: 0.3301 


In [18]:
y_pred = estimator.predict(X_test)
y_pred = [1 if x >= 0.5 else 0 for x in y_pred]
metrics.accuracy_score(y_pred, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step


0.8809523809523809

### **The Functional Model API**

The Sequential class in Keras is typically used for implementing basic neural network architectures. The `Functional` class is typically preferred by most deep learning practitioners as it is more flexible and enables more complex implementations. It allows you to have non-linear topologies, different layers, multiple inputs and outputs. You can easily define branches in your architectures and easily share layers inside architectures. A Functional model is a graph where the nodes of the layers can connect in many more ways than one.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/functional_model.png" alt="The Sequential model API" width="50%">


All models that are implemented in `Sequential()` can easily be implemented in `Functional()`. Some examples of models that are implemented in `Functional()` include ResNet, GoogleNet, Xception. 

Both `Sequential()` and `Functional()` use the same training, evaluation, and inference functions. 

The Model class offers a built in training loop through the `.fit()` method, and a built-in evaluation loop using the `.evaluate()` method. 

The following is a simple example of fitting and evaluating a model on MNIST test data:    


In [19]:
def functional_model():
    inputs = keras.Input(shape=(60,))
    layer1 = Dense(60, activation='relu')(inputs)
    layer2 = Dense(60, activation='relu')(layer1)
    outputs = Dense(1, activation='sigmoid')(layer2)

    model = keras.Model(inputs, outputs)
    
    # Compile model, write code below
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    return model

In [21]:
functional_estimator = functional_model()

In [22]:
functional_estimator.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 60)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 60)                  │           3,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 60)                  │           3,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 1)                   │              61 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,381 (28.83 KB)

 Trainable params: 7,381 (28.83 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
functional_estimator.fit(X_train, y_train, epochs=10, batch_size=16)

Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5555 - loss: 0.6902
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6388 - loss: 0.6606 
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7002 - loss: 0.6611 
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7125 - loss: 0.6438 
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7879 - loss: 0.6257  
Epoch 6/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7167 - loss: 0.6101 
Epoch 7/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7410 - loss: 0.5925 
Epoch 8/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7881 - loss: 0.5554 
Epoch 9/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7711 - loss: 0.5498 
Epoch 10/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7757 - loss: 0.4976 


In [25]:
y_pred_functional = functional_estimator.predict(X_test)
y_pred_functional = [1 if x >= 0.5 else 0 for x in y_pred_functional]
metrics.accuracy_score(y_pred_functional, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


0.8571428571428571

### **Model Subclassing**

Next, we will learn how to make new models and classes using model sub-classing. This method is more flexible and can be used to implement out-of-box models, but this comes at a cost, it is a lot harder to utilize than `Sequential()` and `Functional()` API. 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/subclassing.png" alt="The Sequential model API" width="30%">


We will implement a custom model that performs a type of convolution or pooling. In the following example, we will implement a Sequential model but define it with multiple inputs and outputs. 
Next, we will learn how to make new models and classes using model sub-classing. This method is more flexible and can be used to implement out-of-box models, but this comes at a cost, it is a lot harder to utilize than `Sequential()` and `Functional()` API. 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML311-Coursera/labs/Module1/L1/images/subclassing.png" alt="The Sequential model API" width="30%">


We will implement a custom model that performs a type of convolution or pooling. In the following example, we will implement a Sequential model but define it with multiple inputs and outputs. 


In [27]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = Dense(60, activation='relu')
        self.dense2 = Dense(60, activation='relu')
        self.dense3 = Dense(60, activation='sigmoid')
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(inputs)
        return self.dense3(x)
def sub_class_model():
    inputs = keras.Input(shape=(60,))
    mymodel = MyModel()
    output = mymodel.call(inputs)

    model = keras.Model(inputs, output)
    model.compile(
        optimizer = 'adam',
        loss = 'binary_crossentropy',
        metrics= ['accuracy']
    )
    return model

In [28]:
subclass_estimator = sub_class_model()
subclass_estimator.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)           │ (None, 60)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 60)                  │           3,660 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 60)                  │           3,660 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,320 (28.59 KB)

 Trainable params: 7,320 (28.59 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
subclass_estimator.fit(X_train, y_train, )